In [1]:
import langchain
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    groq_api_key='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',
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [11]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-39383?from=job%20search%20funnel")
page_data=loader.load().pop().page_content
print(page_data)

Apply for Software Engineer II - Workday Integration, ITC

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware Engineer II - Workday Integration, ITCKarnataka, IndiaBecome

In [12]:
from langchain_core.prompts import PromptTemplate
prompt_extract=PromptTemplate.from_template(
    """
    ###Scraped text from website:
    {page_data}
    Instruction:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the following keys:
    'role','experience','skills','description'
    Only return valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)
chain_extract=prompt_extract | llm
res=chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "Software Engineer II - Workday Integration, ITC",
  "experience": "5+ years of experience building integrations and software development",
  "skills": [
    "Workday Integrations with Workday Studio, EIB, Core Connectors and payroll connectors (PECI)",
    "XML, XSLT",
    "Web Services (WSDL, SOAP), EDI and Flat file integrations",
    "SaaS applications, SaaS deployment models, and SaaS DevOps",
    "Object Oriented concepts",
    "Agile Framework"
  ],
  "description": "We’re looking for a Workday Integration Specialist to support HR Services and Operations. This role will sit within our HR Technology Engineering - the team who supports Workday Integration solutions for Core HR Shared Services, Payroll, Total Rewards Operations, Talent Operations, Talent Acquisition, and Learning & Development."
}
```


In [13]:
from langchain_core.output_parsers import JsonOutputParser
json_parser=JsonOutputParser()
json_res=json_parser.parse(res.content)
json_res

{'role': 'Software Engineer II - Workday Integration, ITC',
 'experience': '5+ years of experience building integrations and software development',
 'skills': ['Workday Integrations with Workday Studio, EIB, Core Connectors and payroll connectors (PECI)',
  'XML, XSLT',
  'Web Services (WSDL, SOAP), EDI and Flat file integrations',
  'SaaS applications, SaaS deployment models, and SaaS DevOps',
  'Object Oriented concepts',
  'Agile Framework'],
 'description': 'We’re looking for a Workday Integration Specialist to support HR Services and Operations. This role will sit within our HR Technology Engineering - the team who supports Workday Integration solutions for Core HR Shared Services, Payroll, Total Rewards Operations, Talent Operations, Talent Acquisition, and Learning & Development.'}

In [15]:
import pandas as pd
df=pd.read_csv(r'C:\Users\khale\OneDrive\Desktop\ML\Cold_email_generator\my_portfolio.csv')


In [19]:
import chromadb
import uuid
client=chromadb.PersistentClient('vectorstore')
collection=client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links":row["Links"]},
                       ids=[str(uuid.uuid4())])

In [20]:
job =json_res
links=collection.query(query_texts=job['skills'],n_results=2).get('metadatas',[])

In [21]:
links

[[{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}]]

In [22]:
prompt_email=PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    ###INSTRUCTION:
    You are Khaleel , a business development executive at UNIQ SOLUTIONS.UNIQ SOLUTIONS is an AI & Software
    services company which specializes in seamless integration of business processes through automated tools.
    Over our experience we have empowered numerous enterprises with tailored solutions and processes optimization, cost reduction and heightened overall efficiency
    Your job is to write a cold email to the client regarding the job mentioned above describing the capability in fulfilling their needs.
    Also add most relevant ones from following links to showcase UNIQ SOLUTIONS portfolio : {link_list}.
    ### EMAIL (NO PREAMBLE)
    """
    
)
chain_email=prompt_email|llm
res=chain_email.invoke({"job_description" :str(job),"link_list":links})
print(res.content)

Subject: Expert Workday Integration Solutions for Seamless HR Operations

Dear Hiring Manager,

I came across the job posting for a Software Engineer II - Workday Integration, ITC, and I'm excited to introduce UNIQ SOLUTIONS as a trusted partner in delivering top-notch Workday integration solutions. With our expertise in building integrations and software development, I'm confident that we can support your HR Services and Operations team in achieving their goals.

Our team at UNIQ SOLUTIONS has extensive experience in Workday Integrations using Workday Studio, EIB, Core Connectors, and payroll connectors (PECI). We're well-versed in XML, XSLT, Web Services (WSDL, SOAP), EDI, and Flat file integrations, ensuring seamless connectivity with various SaaS applications. Our expertise in SaaS deployment models and SaaS DevOps enables us to provide efficient and scalable solutions.

We've empowered numerous enterprises with tailored solutions, process optimization, cost reduction, and heighten